# Bornholdt Spin Market Model — Intuitive Overview

The Bornholdt spin model formalizes a financial market as a lattice of interacting traders,  
each able to assume one of two discrete states that represent their trading attitude.  
The model captures complex collective behavior through simple local rules involving **herding**, **contrarian feedback**, and **stochasticity**.

### Model: Step-by-step view

Let each trader $i$ take a spin variable $S_i \in \{+1,-1\}$, where  
$S_i = +1$ denotes a buyer (optimistic agent) and $S_i = -1$ a seller (pessimistic agent).  
At each discrete time step, the state of every trader evolves according to social influence and random fluctuations.

The effective local field acting on trader $i$ is defined as  

$$
h_i = J \sum_{j \in \text{n.n.}} S_j - \alpha S_i |M| ,
$$

where  

- $J$ : herding strength, determining the weight of neighbor imitation,  
- $M = \tfrac{1}{N}\sum_j S_j$ : global magnetization, representing market sentiment,  
- $\alpha$ : global contrarian coupling, describing the pressure to deviate from collective consensus.

The first term, $J\sum S_j$, promotes conformity to local trends,  
while the second term, $-\alpha S_i |M|$, introduces a stabilizing tendency to oppose excessive uniformity.  
Hence, $h_i$ encapsulates the competing social and global influences acting on a trader.

The decision dynamics follow a probabilistic heat–bath rule:

$$
P(S_i = +1) = \frac{1}{1 + e^{-2\beta h_i}}, \qquad
P(S_i = -1) = 1 - P(S_i = +1) ,
$$

where $\beta$ is the inverse temperature controlling sensitivity to $h_i$.  
Large $\beta$ corresponds to deterministic imitation; small $\beta$ introduces randomness.

Each trader draws $u_i \sim U(0,1)$ and updates according to  

$$
S_i =
\begin{cases}
+1, & u_i < P(S_i = +1),\\[4pt]
-1, & \text{otherwise.}
\end{cases}
$$

This stochastic mechanism captures imperfect rationality—agents are influenced by trends and market mood but occasionally act against them due to noise or idiosyncratic behavior.

**Interpretation:**  
The model embodies three intertwined forces: local herding, global contrarianism, and random perturbation.  
Their interaction naturally produces intermittent collective phenomena such as speculative bubbles, sudden crashes, and volatility clustering.


# Market Metrics Observed in Bornholdt (2001)

The paper *Expectation Bubbles in a Spin Model of Markets* (Stefan Bornholdt, 2001) links microscopic spin dynamics to macroscopic financial observables.  
It demonstrates that complex market phenomena emerge from simple local interaction rules.

### 1. Magnetization — Market Sentiment

**Definition:**  
$M(t) = \frac{1}{N} \sum_i S_i(t)$  

**Meaning:**  
$M(t)$ measures the average opinion of all traders, functioning as the *aggregate market sentiment* or *net demand*.  
A positive $M(t)$ indicates a bullish market; a negative value indicates bearish behavior.  

**Role:**  
Acts as a proxy for the market’s overall direction — the balance between buying and selling pressure.

### 2. Returns — Price Changes

**Definition:**  
$r(t) = \ln M(t) - \ln M(t-1)$  

**Meaning:**  
Represents the change in collective sentiment between successive time steps.  
It is directly comparable to the logarithmic return in empirical finance.  

**Observation:**  
The model produces irregular, clustered fluctuations reminiscent of real market price changes.

### 3. Volatility Clustering

**Definition:**  
Examine the autocorrelation of $|r(t)|$, the absolute value of returns.  

**Meaning:**  
Large movements in either direction tend to occur in bursts.  
Calm periods are followed by active ones — volatility is temporally correlated.  

**Observation:**  
This reproduces the *persistence of volatility* seen in actual markets.

### 4. Return Distribution — Fat Tails

**Definition:**  
Analyze the cumulative distribution function (CDF) of $|r(t)|$.  

**Meaning:**  
The decay is slower than Gaussian, indicating heavy tails and an elevated likelihood of extreme returns.  

**Observation:**  
The tails approximate a power-law, aligning with empirical distributions in financial data.

### 5. Intermittency — Bubbles and Crashes

**Definition:**  
Inspect the time series of $M(t)$ and $r(t)$.  

**Meaning:**  
The system exhibits alternating ordered (stable) and disordered (volatile) phases,  
corresponding to endogenous speculative bubbles and crashes.  

**Observation:**  
Boom–bust cycles arise *without external shocks*, purely from internal feedback mechanisms.

### Summary Table

| Metric | Definition | Economic Analogy | Observed Behavior |
|:--|:--|:--|:--|
| $M(t)$ | $\tfrac{1}{N}\sum_i S_i$ | Market sentiment / net demand | Switches between bullish and bearish regimes |
| $r(t)$ | $\ln M(t) - \ln M(t-1)$ | Price return | Irregular, clustered fluctuations |
| $|r(t)|$ autocorr. | — | Volatility measure | Long memory, clustering |
| Return distribution | CDF or histogram of $r(t)$ | Tail risk | Fat tails, power-law decay |
| $M(t)$ series | — | Market phases | Alternating bubbles and crashes |

**In summary:**  
Bornholdt’s spin market model produces empirically realistic patterns — volatility clustering, fat-tailed returns, and endogenous cycles — using only imitation, contrarian feedback, and stochastic decision-making.


In [ ]:
# Focused Bornholdt/Ising visual + live stats + uniform draw panel (with used p & u)
# ----------------------------------------------------------------------------------
# - 10x10 grid with numeric labels only (1 for +1, 0 for -1).
# - Highlights focus cell (5,7) + its 4-neighbor local influence.
# - Non-neighbors tinted by M(t); neighbors by their value; focus by P_used (if available) else P(frame t).
# - Right-hand stats: M(t), P_used (and current-frame estimate), h_focus(t), J*Σ_neighbors S_j(t), constants.
# - Bottom row: Uniform[0,1] panel showing the actual u used for the focus cell during t->t+1.
# - Controls: Play, slider, Next step. 20 steps.

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display, clear_output

# -----------------------------
# Model params
# -----------------------------
N = 10
STEPS = 20
J = 1.0        # herding strength
ALPHA = 6.0    # global contrarian coupling
BETA = 0.8     # inverse temperature
SEED = 7
FOCUS = (5, 7) # (row, col) 0-indexed

rng = np.random.default_rng(SEED)

# -----------------------------
# Ising / Bornholdt helpers
# -----------------------------
def initial_grid(n, rng):
    return rng.choice([-1, 1], size=(n, n))

def magnetization(spins):
    return spins.mean()

def neighbors4(i, j, n):
    return [((i-1) % n, j), ((i+1) % n, j), (i, (j-1) % n), (i, (j+1) % n)]

def neighbors_sum4(spins, i, j):
    s = 0
    n = spins.shape[0]
    for (r, c) in neighbors4(i, j, n):
        s += spins[r, c]
    return s

def heat_bath_sweep(spins, J, alpha, beta, rng, focus_rc=None):
    """
    One asynchronous (random-serial) sweep.
    Returns:
      new_spins, focus_u, focus_h, focus_p
    where focus_* are the u, h, and p_plus when 'focus_rc' was updated during this sweep.
    """
    n = spins.shape[0]
    order = [(i, j) for i in range(n) for j in range(n)]
    rng.shuffle(order)
    M = magnetization(spins)  # use |M| at sweep start

    focus_u = None
    focus_h = None
    focus_p = None

    for (i, j) in order:
        s = spins[i, j]
        local = J * neighbors_sum4(spins, i, j)
        global_term = -alpha * s * abs(M)
        h = local + global_term
        p_plus = 1.0 / (1.0 + np.exp(-2.0 * beta * h))
        u = rng.random()

        if focus_rc is not None and (i, j) == focus_rc:
            focus_u = u
            focus_h = h
            focus_p = p_plus

        spins[i, j] = 1 if u < p_plus else -1

    return spins, focus_u, focus_h, focus_p

def simulate_frames_with_focus_draws(n, steps, J, alpha, beta, seed, focus_rc):
    rng = np.random.default_rng(seed)
    spins = initial_grid(n, rng)

    frames = [spins.copy()]
    M_list = [magnetization(spins)]
    u_focus, h_focus, p_focus = [], [], []

    for _ in range(steps):
        spins, u, h, p = heat_bath_sweep(spins, J, alpha, beta, rng, focus_rc=focus_rc)
        frames.append(spins.copy())
        M_list.append(magnetization(spins))
        u_focus.append(u); h_focus.append(h); p_focus.append(p)

    return (np.asarray(frames), np.asarray(M_list),
            np.asarray(u_focus, dtype=float),
            np.asarray(h_focus, dtype=float),
            np.asarray(p_focus, dtype=float))

frames, M_series, u_focus_next, h_focus_next, p_focus_next = simulate_frames_with_focus_draws(
    N, STEPS, J, ALPHA, BETA, SEED, FOCUS
)

# -----------------------------
# Color helpers (linear red↔white↔green)
# -----------------------------
def mix_red_white_green(x):
    """
    Map x in [-1, 1] to RGB:
      -1 -> red (1,0,0)
       0 -> white (1,1,1)
      +1 -> green (0,1,0)
    """
    x = np.clip(x, -1.0, 1.0)
    if x < 0:
        t = (x + 1.0) / 1.0       # red -> white
        return (1.0, t, t)
    else:
        t = x                     # white -> green
        return (1.0 - t, 1.0, 1.0 - t)

def prob_color(p):
    """
    Map probability p in [0,1] to red→white→green:
      0 -> red, 0.5 -> white, 1 -> green.
    """
    p = float(np.clip(p, 0.0, 1.0))
    if p < 0.5:
        t = p / 0.5               # red -> white
        return (1.0, t, t)
    else:
        t = (p - 0.5) / 0.5       # white -> green
        return (1.0 - t, 1.0, 1.0 - t)

# -----------------------------
# UI / rendering
# -----------------------------
play = W.Play(value=0, min=0, max=len(frames)-1, step=1, interval=300, description="Play")
slider = W.IntSlider(value=0, min=0, max=len(frames)-1, step=1, description='t', layout=W.Layout(width='420px'))
W.jslink((play, 'value'), (slider, 'value'))
btn_next = W.Button(description="Next step (+1)", tooltip="Advance by one step")

# Left: grid output
out_grid = W.Output()

# Right: stats panel (pure widgets; no matplotlib)
stat_title   = W.HTML("<b>Stats</b>")
stat_M       = W.HTML()
stat_P       = W.HTML()   # will show P_used and current-frame estimate
stat_h       = W.HTML()
stat_local   = W.HTML()
stat_consts  = W.HTML()
stat_u_next  = W.HTML()   # actual u used for focus at t->t+1

stats_box = W.VBox([
    stat_title,
    stat_M,
    stat_P,
    stat_h,
    stat_local,
    stat_u_next,
    W.HTML("<hr>"),
    stat_consts
], layout=W.Layout(width='300px'))

# Build the grid figure and close it to avoid extra static display
fig, ax = plt.subplots(figsize=(5.5, 5.5))
ax.set_xlim(-0.5, N-0.5)
ax.set_ylim(N-0.5, -0.5)
ax.set_aspect('equal')
ax.set_xticks(range(N)); ax.set_yticks(range(N))
ax.set_xticklabels([]); ax.set_yticklabels([])
for i in range(N+1):
    ax.axhline(i-0.5, lw=0.5, color='0.8')
    ax.axvline(i-0.5, lw=0.5, color='0.8')

# Pre-create rectangles and text labels
rects = [[None]*N for _ in range(N)]
labels = [[None]*N for _ in range(N)]
for i in range(N):
    for j in range(N):
        r = plt.Rectangle((j-0.5, i-0.5), 1, 1, facecolor=(1,1,1), edgecolor='none', alpha=0.6)
        ax.add_patch(r)
        rects[i][j] = r
        t = ax.text(j, i, '', ha='center', va='center', fontsize=11, fontweight='bold')
        labels[i][j] = t

plt.close(fig)  # avoid extra non-dynamic visuals

# Bottom uniform panel (own figure wrapped in Output)
out_unif = W.Output()
fig_u, ax_u = plt.subplots(figsize=(6.0, 1.6))
ax_u.set_xlim(0, 1)
ax_u.set_ylim(0, 1.1)
ax_u.set_xlabel("u ~ Uniform(0,1)")
ax_u.set_yticks([])
# draw flat PDF = 1 line
ax_u.hlines(1.0, 0, 1, linewidth=2)
# vertical marker line we will update
marker_line = ax_u.vlines(0.0, 0, 1.05, linewidth=2)
title_u = ax_u.set_title("Focus draw for next update (t → t+1): u = N/A")
plt.close(fig_u)

def compute_focus_terms_current_frame(frame, focus_rc):
    """Compute M(t), h_i(t), local_term(t), P_focus(t) based on the displayed frame at time t."""
    i, j = focus_rc
    s = frame[i, j]
    M = frame.mean()
    local_sum = neighbors_sum4(frame, i, j)   # raw neighbor sum
    local_term = J * local_sum                 # J * Σ_j S_j
    global_term = -ALPHA * s * abs(M)
    h = local_term + global_term
    p_plus = 1.0 / (1.0 + np.exp(-2.0 * BETA * h))
    return M, p_plus, h, local_term

def render(t):
    frame = frames[t]
    i0, j0 = FOCUS
    nbs = set(neighbors4(i0, j0, N))

    # Stats based on the *current* frame (frozen t)
    M_t, p_est, h_i, local_term = compute_focus_terms_current_frame(frame, FOCUS)

    # Colors
    bg_col = mix_red_white_green(M_t)

    # Decide probability to color focus cell: use actual used p if available
    if t < len(p_focus_next) and p_focus_next[t] == p_focus_next[t]:
        p_used = float(p_focus_next[t])
    else:
        p_used = p_est  # fallback

    # Update each cell (numbers always visible)
    for i in range(N):
        for j in range(N):
            val = frame[i, j]
            labels[i][j].set_text('1' if val == 1 else '0')

            if (i, j) == (i0, j0):
                rects[i][j].set_facecolor(prob_color(p_used))  # focus: color by p_used
                rects[i][j].set_alpha(0.7)
                labels[i][j].set_color('black')
            elif (i, j) in nbs:
                rects[i][j].set_facecolor(mix_red_white_green(val))  # neighbor: own value color
                rects[i][j].set_alpha(0.65)
                labels[i][j].set_color('black')
            else:
                rects[i][j].set_facecolor(bg_col)  # rest: M-color
                rects[i][j].set_alpha(0.22)
                labels[i][j].set_color('black')

    # Title on grid
    ax.set_title(f"t = {t} | Focus = {FOCUS}")

    # Update right-hand stats
    stat_M.value = f"<b>M(t):</b> {M_t:.4f}"
    if t < len(p_focus_next) and p_focus_next[t] == p_focus_next[t]:
        stat_P.value = (f"<b>P(S<sub>focus</sub>=+1 | used at t→t+1):</b> {p_used:.4f}"
                        f"<br><small>(current-frame estimate: {p_est:.4f})</small>")
    else:
        stat_P.value = (f"<b>P(S<sub>focus</sub>=+1 | used at t→t+1):</b> N/A"
                        f"<br><small>(current-frame estimate: {p_est:.4f})</small>")
    stat_h.value      = f"<b>h<sub>focus</sub>(t):</b> {h_i:.4f}"
    stat_local.value  = f"<b>J·Σ<sub>n.n.</sub>S<sub>j</sub>(t):</b> {local_term:.4f}"
    stat_consts.value = (
        f"<b>Constants</b><br>"
        f"J={J}, α={ALPHA}, β={BETA}<br>"
        f"N={N}, STEPS={STEPS}, SEED={SEED}"
    )

    # --- Uniform panel (actual u used for focus during sweep t) ---
    if t < len(u_focus_next):
        u_val = u_focus_next[t]
        if u_val is not None and not np.isnan(u_val):
            marker_line.set_segments([[(u_val, 0), (u_val, 1.05)]])
            title_u.set_text(f"Focus draw for next update (t → t+1): u = {u_val:.4f}")
            stat_u_next.value = f"<b>u<sub>focus</sub> (for t→t+1):</b> {u_val:.4f}"
        else:
            title_u.set_text("Focus draw for next update (t → t+1): u = N/A")
            stat_u_next.value = f"<b>u<sub>focus</sub> (for t→t+1):</b> N/A"
    else:
        title_u.set_text("Focus draw for next update (t → t+1): u = N/A")
        stat_u_next.value = f"<b>u<sub>focus</sub> (for t→t+1):</b> N/A"

    # Render outputs
    with out_grid:
        clear_output(wait=True)
        display(fig)
    with out_unif:
        clear_output(wait=True)
        display(fig_u)

def on_slider_change(change):
    render(change['new'])

def on_next_clicked(b):
    slider.value = min(slider.value + 1, slider.max)

slider.observe(on_slider_change, names='value')
btn_next.on_click(on_next_clicked)

# Initial draw & layout
render(0)
controls = W.HBox([play, slider, btn_next])
top_row = W.HBox([out_grid, stats_box])
bottom_row = W.HBox([out_unif])
display(W.VBox([controls, top_row, bottom_row]))
